In [24]:
# Importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,  r2_score
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers


In [25]:
# Loading the dataset
# Train Data
temp_df = pd.read_csv("../../data/cleaned/train.csv", nrows=0)  # Read only the header
total_columns = len(temp_df.columns)
columns_to_use = temp_df.columns[1:total_columns]
train_data = pd.read_csv("../../data/cleaned/train.csv",  usecols=columns_to_use)
print(train_data.head(5))
# Test Data
test_data = pd.read_csv("../../data/cleaned/test.csv", usecols=columns_to_use)
print(test_data.head())

            building_name        meter        date  meter_reading site_name  \
0  Bear_education_Alfredo  electricity  2016-01-01         2.9050      Bear   
1  Bear_education_Alfredo  electricity  2016-01-02         2.7700      Bear   
2  Bear_education_Alfredo  electricity  2016-01-03         2.6725      Bear   
3  Bear_education_Alfredo  electricity  2016-01-04         4.5650      Bear   
4  Bear_education_Alfredo  electricity  2016-01-05         4.7825      Bear   

  sub_primaryspaceusage    sqm    sqft    timezone  airTemperature  \
0             Education  609.8  6564.0  US/Pacific        5.246861   
1             Education  609.8  6564.0  US/Pacific        5.993973   
2             Education  609.8  6564.0  US/Pacific        5.660314   
3             Education  609.8  6564.0  US/Pacific        5.048507   
4             Education  609.8  6564.0  US/Pacific        4.745567   

   cloudCoverage  dewTemperature  precipDepth1HR  precipDepth6HR  \
0       1.927009        0.254484    

In [26]:
test_data['meter'].unique()

array(['electricity', 'chilledwater', 'gas', 'hotwater', 'solar', 'water',
       'steam', 'irrigation'], dtype=object)

In [27]:
# Dropping the columns that are not relevant to our analysis
train_data = train_data.drop(columns=['building_name', 'site_name'])
test_data = test_data.drop(columns=['building_name', 'site_name'])

train_data = train_data[(train_data['meter'] == 'electricity') | (train_data['meter'] == 'chilledwater') | (train_data['meter'] == 'steam')
| (train_data['meter'] == 'hotwater') | (train_data['meter'] == 'gas')]

test_data = test_data[(test_data['meter'] == 'electricity') | (test_data['meter'] == 'chilledwater') | (test_data['meter'] == 'steam')
| (test_data['meter'] == 'hotwater') | (test_data['meter'] == 'gas')]

# Building index on building_id for further assessment
train_data.set_index('building_id', inplace=True)
test_data.set_index('building_id', inplace=True)

In [28]:
# Filtering for hotwater meter_reading
train_data = train_data[train_data['meter'] == 'hotwater']
test_data = test_data[test_data['meter'] == 'hotwater']

train_data = train_data.drop(columns=['meter'])
test_data = test_data.drop(columns=['meter'])

In [29]:
# Inspecting the data frames
print(train_data.sample(2))
print('-------------------------------------------------------------')
print(test_data.sample(2))

                   date  meter_reading sub_primaryspaceusage      sqm  \
building_id                                                             
584          2016-09-07          0.000    College Laboratory   2992.0   
78           2016-04-29        492.697              Academic  16116.7   

                 sqft       timezone  airTemperature  cloudCoverage  \
building_id                                                           
584           32206.0  Europe/London       23.652193       1.860015   
78           173479.0    US/Mountain       12.281978       2.105182   

             dewTemperature  precipDepth1HR  precipDepth6HR  seaLvlPressure  \
building_id                                                                   
584               16.769737        1.202212       11.328716     1015.793681   
78                 5.268132        0.406701       12.112282     1014.526752   

             windDirection  windSpeed  season  site_id  
building_id                                     

In [30]:
# Separating into X and Y dataframes
X_train = train_data.drop(columns=['meter_reading'])  # Exclude target variable
y_train = train_data['meter_reading']

X_test = test_data.drop(columns=['meter_reading'])  # Exclude target variable
y_test = test_data['meter_reading']

In [31]:
# Convert 'site_id' from numeric to categorical
X_train['site_id'] = X_train['site_id'].astype('category')
X_test['site_id'] = X_test['site_id'].astype('category')

In [32]:
print(X_train.dtypes)
print(X_train.columns)

date                       object
sub_primaryspaceusage      object
sqm                       float64
sqft                      float64
timezone                   object
airTemperature            float64
cloudCoverage             float64
dewTemperature            float64
precipDepth1HR            float64
precipDepth6HR            float64
seaLvlPressure            float64
windDirection             float64
windSpeed                 float64
season                     object
site_id                  category
dtype: object
Index(['date', 'sub_primaryspaceusage', 'sqm', 'sqft', 'timezone',
       'airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR',
       'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed',
       'season', 'site_id'],
      dtype='object')


In [33]:
# Define features and types based on your dataset
numerical_features = ['sqm', 'sqft', 'airTemperature', 'cloudCoverage', 'dewTemperature',
                      'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']
categorical_features = ['timezone', 'season', 'sub_primaryspaceusage', 'site_id']

In [34]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [35]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Fit the preprocessor on the training data and transform both training and test data
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [36]:
# Convert the processed data back to dense DataFrames
X_train_processed_df = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out())
X_test_processed_df = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out())

In [37]:
# Checking the columns
X_train_processed_df.columns

Index(['num__sqm', 'num__sqft', 'num__airTemperature', 'num__cloudCoverage',
       'num__dewTemperature', 'num__precipDepth1HR', 'num__precipDepth6HR',
       'num__seaLvlPressure', 'num__windDirection', 'num__windSpeed',
       'cat__timezone_Europe/London', 'cat__timezone_US/Eastern',
       'cat__timezone_US/Mountain', 'cat__season_Fall', 'cat__season_Spring',
       'cat__season_Summer', 'cat__season_Winter',
       'cat__sub_primaryspaceusage_Academic',
       'cat__sub_primaryspaceusage_Classroom',
       'cat__sub_primaryspaceusage_College Classroom',
       'cat__sub_primaryspaceusage_College Laboratory',
       'cat__sub_primaryspaceusage_Research',
       'cat__sub_primaryspaceusage_Student Center',
       'cat__sub_primaryspaceusage_Student Union', 'cat__site_id_2',
       'cat__site_id_4', 'cat__site_id_5', 'cat__site_id_6', 'cat__site_id_7',
       'cat__site_id_10', 'cat__site_id_14'],
      dtype='object')

In [38]:
y_train_scaled = np.log1p(y_train.values.reshape(-1, 1))
y_test_scaled = np.log1p(y_test.values.reshape(-1, 1))

### Neural Networks

In [39]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_processed_df.shape[1], activation='relu')) # Adjust input_dim based on your features
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer, adjust units and activation based on your output

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [40]:
# Fit the model to the training data
model.fit(X_train_processed, y_train_scaled, epochs=10, batch_size=10, verbose=1)

# Predict the values for X_train
y_pred = model.predict(X_train_processed_df)

Epoch 1/10
2965/2965 [==============================] - 1s 322us/step - loss: 19.4507
Epoch 2/10
2965/2965 [==============================] - 1s 313us/step - loss: 17.1027
Epoch 3/10
2965/2965 [==============================] - 1s 306us/step - loss: 16.4448
Epoch 4/10
2965/2965 [==============================] - 1s 310us/step - loss: 15.8546
Epoch 5/10
2965/2965 [==============================] - 1s 309us/step - loss: 15.1307
Epoch 6/10
2965/2965 [==============================] - 1s 310us/step - loss: 14.5777
Epoch 7/10
2965/2965 [==============================] - 1s 313us/step - loss: 14.1444
Epoch 8/10
2965/2965 [==============================] - 1s 322us/step - loss: 13.8370
Epoch 9/10
2965/2965 [==============================] - 1s 308us/step - loss: 13.5134
Epoch 10/10
927/927 [==============================] - 0s 211us/step


In [41]:
#mae = mean_absolute_error(y_train_scaled, y_pred)
mse = mean_squared_error(y_train_scaled, y_pred)
r2 = r2_score(y_train_scaled, y_pred)

#print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)


Mean Squared Error: 13.303919684359082
R-squared Score: 0.5726096069162547


In [42]:
# Predict the values for X_test
y_pred = model.predict(X_test_processed_df)
y_pred = np.nan_to_num(y_pred, nan=0)
mse = mean_squared_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)

924/924 [==============================] - 0s 223us/step
Mean Squared Error: 18.02956578787283
R-squared Score: 0.4691722820386899


In [43]:

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(
        units=hp.Int('input_units', min_value=32, max_value=256, step=64),  # Reduced max value and increased step
        activation='relu',
        input_shape=(X_train_processed.shape[1],)
    ))

    # Reduced the maximum number of layers
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'layer_{i}_units', min_value=32, max_value=256, step=64),  # Simplify layers
            activation='relu'
        ))

    model.add(layers.Dense(1)) 

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),  # Simplified choices
        loss='mse',  
        metrics=['mae', 'mse'] 
    )

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_mse', 
    max_trials=10,  
    executions_per_trial=2,  
    directory='my_dir',
    project_name='quick_tune'
)

tuner.search(
    X_train_processed_df, y_train_scaled,
    epochs=5,  # Reduced number of epochs for tuning
    validation_split=0.1,  # Adjust based on your dataset
    batch_size=256  # Increased batch size for faster processing
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train_processed_df, y_train_scaled,
    epochs=10,  # You can increase this for the final model training
    validation_split=0.1,  # Keep consistent with tuning phase or adjust as necessary
    batch_size=256  # Set a fixed batch size
)

Reloading Tuner from my_dir/quick_tune/tuner0.json


Epoch 1/10
105/105 [==============================] - 0s 1ms/step - loss: 79.9790 - mae: 7.3676 - mse: 79.9790 - val_loss: 29.8135 - val_mae: 5.0782 - val_mse: 29.8135
Epoch 2/10
105/105 [==============================] - 0s 847us/step - loss: 46.9201 - mae: 6.0950 - mse: 46.9201 - val_loss: 15.0704 - val_mae: 3.6573 - val_mse: 15.0704
Epoch 3/10
105/105 [==============================] - 0s 829us/step - loss: 29.7073 - mae: 4.9044 - mse: 29.7073 - val_loss: 14.2035 - val_mae: 3.5895 - val_mse: 14.2035
Epoch 4/10
105/105 [==============================] - 0s 827us/step - loss: 24.1773 - mae: 4.2865 - mse: 24.1773 - val_loss: 17.3283 - val_mae: 3.9551 - val_mse: 17.3283
Epoch 5/10
105/105 [==============================] - 0s 830us/step - loss: 21.6298 - mae: 3.8747 - mse: 21.6298 - val_loss: 22.1526 - val_mae: 4.3887 - val_mse: 22.1526
Epoch 6/10
105/105 [==============================] - 0s 814us/step - loss: 20.7758 - mae: 3.6920 - mse: 20.7758 - val_loss: 26.3128 - val_mae: 4.7338 -

In [44]:
# Check and replace NaN values in the test features 
if X_test_processed_df.isna().any().any(): 
    X_test_processed_df = X_test_processed_df.fillna(0)  # Replace NaN with 0 

# Check and replace NaN values in the test labels 
if np.isnan(y_test_scaled).any():  
    y_test_scaled = np.nan_to_num(y_test_scaled)  # Replace NaN with 0 (default)

In [45]:
test_loss, test_mae, test_mse = model.evaluate(X_test_processed_df, y_test_scaled, batch_size=256)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}, Test MSE: {test_mse}')

116/116 [==============================] - 0s 374us/step - loss: 21.4778 - mae: 3.7687 - mse: 21.4778
Test Loss: 21.477813720703125, Test MAE: 3.768683671951294, Test MSE: 21.477813720703125


In [46]:
y_pred = model.predict(X_test_processed_df, batch_size=256)

# Calculate R-squared score
r_squared = r2_score(y_test_scaled, y_pred)
print(f'R-squared Score: {r_squared}')

116/116 [==============================] - 0s 342us/step
R-squared Score: 0.36764879776502923
